In [1]:
from itertools import product
import pandas as pd
from mip import Model, xsum, BINARY

case = 2
resource_sheet = "surgery_resources{0}".format(case)
jobs_sheet = "surgery_scen1"


#### Input Data

In [2]:
resource_constraints = pd.read_excel(
    "rcs_data.xlsx", sheet_name=resource_sheet, engine="openpyxl"
)

jobs = pd.read_excel("rcs_data.xlsx", sheet_name=jobs_sheet, engine="openpyxl")
jobs = jobs.set_index("job_id")
print(jobs)

# Number of non-dummy jobs
n = jobs.shape[0] - 2
print("Number of non-dummy jobs:", n, "\n")

# Job duration
p = jobs["estimated_duration"].astype(int).tolist()
p = [int(i) for i in p]
print("Job Duration:", p)
print(len(p), "\n")


# Resource Requirements
u = jobs.iloc[:, 6:].fillna(0).values.astype(int).tolist()
print("Resource Requirements:", u)
print(len(u[0]), len(u), "\n\n")

# Resource Constraints
c = resource_constraints["total_available"].tolist()
print("Resource Constraints", c)
print(len(c), "\n")


# Precedence Constraints
S = []
for i in jobs.index:
    if i != 0:
        preds = str(jobs.loc[i, "precedence"]).split(",")
        for j in preds:
            S.append([int(j), i])
print("Precedence Constraints",S)
print(len(S),"\n")
            
(R, J, T) = (range(len(c)), range(len(p)), range(sum(p)))
print(R,J,T)

        case_id     diagnosis  priority           job_name  \
job_id                                                       
0           NaN           NaN         1        dummy_start   
1          11.0  hip_fracture         2          bloodtest   
2          11.0  hip_fracture         2        hip_surgery   
3          23.0    arthiritis         2          bloodtest   
4          23.0    arthiritis         2       knee_surgery   
5          48.0           NaN         1  breast_transplant   
6           NaN           NaN         1          dummy_end   

        estimated_duration precedence  SeniorOrthopaedicSurgeon  \
job_id                                                            
0                        0        NaN                       NaN   
1                        5          0                       NaN   
2                       75          1                       1.0   
3                        5          0                       NaN   
4                      103          3  

#### Initiate model, add decision variables
$x_{jt}$ : job j starts at time t (binary variable 1 or 0)

In [3]:
model = Model()

x = [
    [model.add_var(name="x({},{})".format(j, t), var_type=BINARY) for t in T] for j in J
]

#### Constraints

In [4]:
# job j can ONLY be completed once, so it can only be assigned to one t
for j in J:
    model += xsum(x[j][t] for t in T) == 1

# For all t, total usage of resource r have to be less than resource constraint
# resource usage = # of resource r required for job j    x    assignment of job to time t2
# for all the time from (t2 = t - job duration) ===> t, if job j starts at t2 then
# the resource usage for that whole period must be less than constraint
print(c)
for (r, t) in product(R, T):
    model += (
        xsum(u[j][r] * x[j][t2] for j in J for t2 in range(max(0, t - p[j] + 1), t + 1))
        <= c[r]
    )

## Precedence Constraint, if j precedes s, 
## then:  time taken to go from job j to job s must be >= processing time job j
for (j, s) in S:
    model += xsum(t * x[s][t] - t * x[j][t] for t in T) >= p[j]


[2, 3, 1, 3, 2, 6, 3, 3, 3, 5, 5, 3, 6, 2]


#### Define Objective and Solve

In [5]:
##### Model Objective can be adjusted.
##### Total time to complete all jobs are minimized 
# # Minimize wait time of preceding/successive events, and makespan, and start job sooner rather than later
### Scale objectives based on relative importance
### in this case ... wait time is most important so it has the highest costs
model.objective = (xsum(t * (x[s][t] - x[j][t])+t * x[n + 1][t]/10 + t/10 * x[j][t] for t in T for (j,s) in S if j !=0 and s != (n+1)))
status = model.optimize()
print(status)

results = {}
print("Schedule: ")
for (j, t) in product(J, T):
    if x[j][t].x >= 0.99 and j!=0 and j!=n+1:
        results[j] = [t,t+p[j]]
        print("Job {}: begins at t={} and finishes at t={}".format(j, t, t + p[j]))

Welcome to the CBC MILP Solver 
Version: Trunk
Build Date: Oct 24 2021 

Starting solution of the Linear programming relaxation problem using Primal Simplex

Clp0024I Matrix will be packed to eliminate 237244 small elements
Coin0506I Presolve 2472 (-1379) rows, 1916 (-2) columns and 352994 (-303286) elements
Clp0030I 13 infeas 0.96750201, obj 17.200053 - mu 0.02870833, its 105, 1421 interior
Clp0030I 21 infeas 0.0038113004, obj 31.601802 - mu 0.0031891766, its 105, 1228 interior
Clp0030I 29 infeas 0.00018749576, obj 31.606683 - mu 0.00011808222, its 105, 1361 interior
Clp0030I 37 infeas 0.0001774132, obj 31.604274 - mu 4.3721036e-06, its 105, 1350 interior
Clp0030I 46 infeas 6.4012823e-06, obj 31.606588 - mu 1.6188119e-07, its 105, 1349 interior
Clp1000I sum of infeasibilities 4.3285e-05 - average 1.75101e-08, 634 fixed columns
Coin0506I Presolve 2330 (-142) rows, 1282 (-634) columns and 340349 (-12645) elements
Clp0006I 0  Obj 31.613219 Primal inf 3.7177635e-05 (3) Dual inf 4.4475552e

## Assign resources to jobs (taking into account the time element)

In [6]:
pool = pd.read_excel(
    "rcs_data.xlsx", sheet_name='resource_pool', engine="openpyxl"
)

assigned = {}
scenario = jobs.copy()
for r in scenario.index:
    if r != 0 and r!=n+1:
        temp_pool = pool.copy()
        if assigned:
            time_overlapped = []
            for j in assigned.keys():
                timeframe_j = range(results[j][0],results[j][1])
                timeframe_r = range(results[r][0],results[r][1])
                # Overlap, need to disable those resources that have been assigned
                if list(set(timeframe_j) & set(timeframe_r)):
                    for key,val in assigned[j].items():
                        temp_pool.loc[(temp_pool['resource']==val)&(temp_pool['unique_identification']==key),'resource'] = 'used'
                    
        a = {}
        for c in scenario.iloc[:, 6:].columns:
            if scenario.loc[r,c]!=0:
                while scenario.loc[r,c]>0:
                    p = temp_pool.loc[temp_pool['resource']==c,'unique_identification'].tolist()[0]
                    temp_pool.loc[(temp_pool['resource']==c)&(temp_pool['unique_identification']==p),'resource'] = 'used'
                    scenario.loc[r,c] -= 1
                    a[p] = c 
        assigned[r]=a
for k,v in assigned.items():
    print(k,v,"\n")

1 {'Becky Neunman': 'PhlebotomyTechnician'} 

2 {'Jaben Forbes': 'SeniorOrthopaedicSurgeon', 'Josiah Ngai': 'ResidentOrthopaedicSurgeon', 'Ruby Miles': 'OrthopaedicNurse', 'Jen Newton': 'OrthopaedicNurse', 'Alicia Keys': 'SurgicalTechnician', 'Julie Watson': 'Anesthesiologist ', 'OperatingRoom #1': 'OperatingRoom', 'SurgicalTable #4': 'SurgicalTable', 'PatientMonitor #7': 'PatientMonitor', 'NasalCannula #12': 'NasalCannula', 'Ceiling-mountedSurgicalLight #17': 'Ceiling-mountedSurgicalLight', 'SurgicalHeadlight #20': 'SurgicalHeadlight', 'SurgicalHeadlight #21': 'SurgicalHeadlight'} 

3 {'Janett Newton': 'PhlebotomyTechnician'} 

4 {'Matt Sell': 'SeniorOrthopaedicSurgeon', 'Oliver Clark': 'ResidentOrthopaedicSurgeon', 'Rob Lokhart': 'ResidentOrthopaedicSurgeon', 'Lauren Flutey': 'OrthopaedicNurse', 'Anton Pade': 'SurgicalTechnician', 'Jensen Murdock': 'SurgicalTechnician', 'Ben Davies': 'Anesthesiologist ', 'OperatingRoom #2': 'OperatingRoom', 'SurgicalTable #5': 'SurgicalTable', 'Patie